## Processing to collect Canada Postal Codes

In [4]:
## Import all related materials

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import geocoder
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import geopy.geocoders # convert an address into latitude and longitude values
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')


In [9]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [10]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [11]:
Geocoder_package=pd.read_csv('https://cocl.us/Geospatial_data')
Geocoder_package.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
Geocoder_package=Geocoder_package.rename(index=str, columns={"Postal Code":"PostalCode"})
Geocoder_package.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
toronto_dict = {'PostalCode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}

In [14]:
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1V,Scarborough,"Agincourt North, Milliken"
1,M8W,Etobicoke,"Alderwood, Long Branch"
2,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
3,M1W,Scarborough,Steeles West
4,M1M,Scarborough,"Cliffcrest, Cliffside"


## Toronto Postal Codes and Neighbourhood

In [15]:
result=pd.merge(Geocoder_package,df_toronto,on='PostalCode')
result=result[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
result.to_csv('toronto_base.csv')
result.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Explore the overview of Toronto

In [108]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## Create a new data frame with neighborhoods in Scarborough

In [18]:
Scarboroughs = result[result['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarboroughs.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a map of Scarborough

In [19]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [109]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Scarboroughs['Latitude'], Scarboroughs['Longitude'], Scarboroughs['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

## Calling details of neighborhood in Scarboroughin by Foursquare

In [36]:
CLIENT_ID = '1RCC5Q1JCPIEJ5O1BGHVKPABSLXWETBNEJUUK54MH4CNGM0Y' # your Foursquare ID
CLIENT_SECRET = 'SD44P3NM3WV1H0SZOHE04M2ZYEB1YK2IZTJMYF0HULE3CCCD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
radius = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1RCC5Q1JCPIEJ5O1BGHVKPABSLXWETBNEJUUK54MH4CNGM0Y
CLIENT_SECRET:SD44P3NM3WV1H0SZOHE04M2ZYEB1YK2IZTJMYF0HULE3CCCD


In [37]:
neighborhood_latitude = Scarboroughs.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = Scarboroughs.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = Scarboroughs.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Rouge, Malvern" are 43.806686299999996, -79.19435340000001.


In [38]:
outcome = requests.get(url).json()
outcome

{'meta': {'code': 200, 'requestId': '5bdea1161ed21972419845d7'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 62,
  'suggestedBounds': {'ne': {'lat': 43.78207700900001,
    'lng': -79.24533335909429},
   'sw': {'lat': 43.76407699099999, 'lng': -79.2702146409057}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
      

In [39]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = outcome['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,Canyon Creek Chophouse,Steakhouse,43.776959,-79.261694
2,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
3,DAVIDsTEA,Tea Room,43.776613,-79.258516
4,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
5,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
6,Shoppers Drug Mart,Pharmacy,43.772747,-79.251123
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,SEPHORA,Cosmetics Shop,43.775592,-79.258242
9,CANBE Foods Inc,Indian Restaurant,43.773546,-79.246082


In [45]:
nearby_venues['categories'].unique()

array(['Toy / Game Store', 'Steakhouse', 'Fish & Chips Shop', 'Tea Room',
       'Mexican Restaurant', 'Clothing Store', 'Pharmacy',
       'Movie Theater', 'Cosmetics Shop', 'Indian Restaurant',
       'Coffee Shop', 'Shopping Mall', 'Gym', 'Sporting Goods Shop',
       'Wings Joint', 'Supermarket', 'Restaurant', 'Department Store',
       'American Restaurant', 'Discount Store', 'Italian Restaurant',
       'Pizza Place', 'Food Court', 'Sandwich Place', 'Smoothie Shop',
       'Bank', 'Grocery Store', 'Thai Restaurant', 'Big Box Store',
       'Fast Food Restaurant', 'Bakery', 'Video Game Store',
       'Yoga Studio', "Men's Store", 'Electronics Store',
       'Storage Facility', 'Deli / Bodega', "Women's Store", 'Road'],
      dtype=object)

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
scarborough_venues = getNearbyVenues(names=Scarboroughs['Neighbourhood'],
                                   latitudes=Scarboroughs['Latitude'],
                                   longitudes=Scarboroughs['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside
Birch Cliff
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Tam O'Shanter
Agincourt North, Milliken
Steeles West
Upper Rouge


In [50]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant
3,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant


In [62]:
#Explore unique valuevaluee of Venue Category
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 117 uniques categories.
Here is the list of different categories:


['Spa',
 'Caribbean Restaurant',
 'Fast Food Restaurant',
 'Paper / Office Supplies Store',
 'Coffee Shop',
 'Hobby Shop',
 'Bus Station',
 'Construction & Landscaping',
 'Martial Arts Dojo',
 'Chinese Restaurant',
 'Greek Restaurant',
 'Fruit & Vegetable Store',
 'Gym',
 'Bakery',
 'Arts & Crafts Store',
 'Sandwich Place',
 'Burger Joint',
 'Italian Restaurant',
 'Breakfast Spot',
 'Playground',
 'Fried Chicken Joint',
 'Food & Drink Shop',
 'Liquor Store',
 'Smoothie Shop',
 'Pizza Place',
 'Beer Store',
 'Discount Store',
 'Pharmacy',
 'Bank',
 'Sports Bar',
 'Supermarket',
 'Park',
 'Indian Restaurant',
 'Electronics Store',
 'Hakka Restaurant',
 'Athletics & Sports',
 'Music Store',
 'Thai Restaurant',
 'Wings Joint',
 'Yoga Studio',
 'Grocery Store',
 'Asian Restaurant',
 'Gym / Fitness Center',
 'Rental Car Location',
 'Bus Line',
 'Sporting Goods Shop',
 'Ice Cream Shop',
 "Women's Store",
 'Restaurant',
 'Convenience Store',
 'Train Station',
 'Japanese Restaurant',
 'Bowling 

In [72]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Music Store,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
# List of Gym center and realted:
target_category=['Neighborhood',
 'Gym',
 'Park',
 'Yoga Studio',
 'Gym / Fitness Center',
 'Soccer Field',
 'Beach',
 'Gym Pool',
 'Badminton Court',
 'Pool',
 'Pool Hall',
 'Golf Course',
 'Tennis Court']

In [81]:
scarb_onehot = scarb_onehot[target_category].groupby('Neighborhood').sum()
scarb_onehot.head()

,Gym,Park,Yoga Studio,Gym / Fitness Center,Soccer Field,Beach,Gym Pool,Badminton Court,Pool,Pool Hall,Golf Course,Tennis Court
Neighborhood,,,,,,,,,,,,
Agincourt,0,1,0,0,0,0,0,1,1,1,0,0
"Agincourt North, Milliken",1,2,0,0,0,0,0,0,0,0,0,0
Birch Cliff,1,2,0,0,0,0,1,0,0,0,0,0
Cedarbrae,0,0,1,1,0,0,0,0,0,0,0,0
"Clairlea, Golden Mile, Oakridge",1,2,0,0,1,0,0,0,0,0,0,0


## Run k-means to cluster the neighborhoods into 5

In [103]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarb_onehot)

In [105]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarb_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Gym,Park,Yoga Studio,Gym / Fitness Center,Soccer Field,Beach,Gym Pool,Badminton Court,Pool,Pool Hall,Golf Course,Tennis Court,Total Sum
G5,0.0,1.000,0.000,0.00,0.000000,0.0,0.000000,1.000,1.0,1.0,0.000,0.000000,4.000000
G1,1.0,2.000,0.000,0.00,0.333333,0.0,0.333333,0.000,0.0,0.0,0.000,0.000000,3.666667
G3,0.0,1.000,0.000,0.00,0.000000,2.0,0.000000,0.000,0.0,0.0,0.000,0.000000,3.000000
G4,1.0,0.000,0.000,0.00,0.000000,0.0,0.333333,0.000,0.0,0.0,0.000,0.333333,1.666667
G2,0.0,0.375,0.125,0.25,0.125000,0.0,0.000000,0.125,0.0,0.0,0.125,0.000000,1.125000


## Summary for neighborhood and Cluster

In [106]:
neigh_summary = pd.DataFrame([scarb_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,5
1,"Agincourt North, Milliken",1
2,Birch Cliff,1
3,Cedarbrae,2
4,"Clairlea, Golden Mile, Oakridge",1
5,"Cliffcrest, Cliffside",3
6,"Dorset Park, Scarborough Town Centre, Wexford ...",2
7,"Highland Creek, Rouge Hill, Port Union",2
8,"Ionview, Kennedy Park",2
9,"Maryvale, Wexford",2


# Thanks for reading